In [3]:
from setup import D

synth_count = 10
d = D(
    synth_count=synth_count,
    context='opl',
)


In [4]:
from enum import Enum
import time


class Event():
    def __init__(self, note, scale, timing, dur, synth_idx):
        self.note = note
        self.scale = scale
        self.timing = timing
        self.dur = dur
        self.synth_idx = synth_idx

class Notes(str, Enum):
    REST = '.'

class Events():
    def __init__(self, stream, scale):
        # NOTE:
        # - scale is global
        # - notation for duration is not finalized
        # - sustain is same as duration
        # - bpm is global
        # - no info about synth indices
        self.events = []
        cur_beat = 0
        for notes in stream:
            # convert to list
            chord = []
            dur = 1
            if isinstance(notes, str) and (notes != Notes.REST):
                notes, dur = notes.split(':')
                dur = int(dur)
                notes = eval(notes)
            match notes:
                case int():
                    chord = [notes]
                case Notes.REST:
                    chord = [Notes.REST]
                case tuple():
                    chord = list(notes)
                case _:
                    print(f'Wrong type for notes {notes}: {type(notes)}')
                    assert False

            for note_idx, note in enumerate(chord):
                self.events.append(Event(note, scale, cur_beat, dur,
                                         synth_idx=note_idx))
            cur_beat += dur


def play(events: Events, client,
         dur=0.25,
         sus=None,
         voice_amps=[],
         # FIXME:
         # delay=None,
         # synth_idx=[0],
         # rep=1,
         ):

    # NOTE: crutch; play random note silently, so that the timbre doesn't jump on first note
    # FIXME: depends on synth
    # timetag = time.time()
    # for note_idx in range(synth_count):
    #     client.send_note(
    #             note_idx, freq=C4_FREQ,
    #             dur=0.1, amp=0.0,
    #             timetag=timetag, channel=0)
    # time.sleep(0.1)

    # FIXME:
    # if isinstance(rep, list):
    #     assert len(synth_idx) == len(rep)
    # else:
    #     rep = [rep] * len(synth_idx)
    # last_reps = [0] * len(synth_idx)

    cur_beat = 0
    for event in events.events:
        assert event.synth_idx < synth_count

        # NOTE: assuming events are sorted by timing
        assert cur_beat <= event.timing
        if event.timing > cur_beat:
            sleep_dur = dur * (event.timing - cur_beat)
            time.sleep(sleep_dur)
            cur_beat = event.timing

        send_note_dur = dur
        if sus:
            send_note_dur = sus
        send_note_dur *= event.dur

        amp = 1.0
        if voice_amps:
            amp *= voice_amps[event.synth_idx]  # TODO: should be voice_idx

        freq = event.scale.get_freq(event.note)
        timetag = time.time()
        client.send_note(
            # FIXME: synth_idx[note_idx] + last_reps[note_idx],
            event.synth_idx,
            freq=freq,
            dur=send_note_dur,
            amp=amp,
            timetag=timetag,
            channel=0,  # TODO: for MPE with channels use channel=note_idx
            )
        # FIXME:
        # last_reps[note_idx] = (last_reps[note_idx] + 1) % rep[note_idx]
        # if delay:
            # time.sleep(delay)
    time.sleep(dur)


In [11]:
from domblar.ed import ED
scale = ED(14)

for i in range(8):
    d.set_synth(i, 'jpiano')

notes = [
    # (0, 4, 8),
    # (0, 3, 8),
    # (0, 5, 8),
    # (0, 4, 7),
    (0, 5, 9),
]

# 0-4-8 = C E G = C = C or C perfect
# 0-3-8 = C vE G = Cv = C down
# 0-5-8 = C ^E G = C^ = C up
# 0-4-7 = C E vG = C(v5) = C down-five
# 0-5-9 = C ^E ^G = C^(^5) = C up up-five

for _ in range(1):
    play(Events(notes, scale=scale),
        d.client,
        dur=3/2/2/2,
        )


In [11]:
# Piano Phase analysis
from domblar.ed import ED

scale = ED(12)
phrase1 = [4, 11, 14] * 2  # steps: 7, 10
phrase2 = [6, 13] * 3  # steps: 2, 7

melody = []
for i in range(len(phrase1)):
    melody.append(phrase1[i])
    melody.append(phrase2[i])

for shift in range(12):
    notes = []
    for i1 in range(len(melody)):
        i2 = (i1 + shift) % len(melody)
        notes.append((melody[i1], melody[i2]))

    intervals = set()
    for chord in notes:
        intervals.add(abs(chord[0] - chord[1]))
    print(shift, intervals)
    print(shift, notes)

# 1,2,5,8,9
# or 0, 3, 7, 10 = kind-of min7 chord?

0 {0}
0 [(4, 4), (6, 6), (11, 11), (13, 13), (14, 14), (6, 6), (4, 4), (13, 13), (11, 11), (6, 6), (14, 14), (13, 13)]
1 {1, 2, 5, 8, 9}
1 [(4, 6), (6, 11), (11, 13), (13, 14), (14, 6), (6, 4), (4, 13), (13, 11), (11, 6), (6, 14), (14, 13), (13, 4)]
2 {10, 3, 7}
2 [(4, 11), (6, 13), (11, 14), (13, 6), (14, 4), (6, 13), (4, 11), (13, 6), (11, 14), (6, 13), (14, 4), (13, 6)]
3 {1, 2, 5, 8, 9}
3 [(4, 13), (6, 14), (11, 6), (13, 4), (14, 13), (6, 11), (4, 6), (13, 14), (11, 13), (6, 4), (14, 6), (13, 11)]
4 {0, 10, 3, 7}
4 [(4, 14), (6, 6), (11, 4), (13, 13), (14, 11), (6, 6), (4, 14), (13, 13), (11, 4), (6, 6), (14, 11), (13, 13)]
5 {1, 2, 5, 8, 9}
5 [(4, 6), (6, 4), (11, 13), (13, 11), (14, 6), (6, 14), (4, 13), (13, 4), (11, 6), (6, 11), (14, 13), (13, 14)]
6 {0, 7}
6 [(4, 4), (6, 13), (11, 11), (13, 6), (14, 14), (6, 13), (4, 4), (13, 6), (11, 11), (6, 13), (14, 14), (13, 6)]
7 {1, 2, 5, 8, 9}
7 [(4, 13), (6, 11), (11, 6), (13, 14), (14, 13), (6, 4), (4, 6), (13, 11), (11, 13), (6, 14)

In [13]:
# Piano Phase, but with chords
from domblar.ed import ED

scale = ED(12)
phrase1 = [4, 11, 14] * 2  # steps: 7, 10
phrase2 = [6, 13] * 3  # steps: 2, 7

melody = []
for i in range(len(phrase1)):
    melody.append(phrase1[i])
    melody.append(phrase2[i])

shift1 = 1
shift2 = 4

notes = []
for i1 in range(len(melody)):
    i2 = (i1 + shift1) % len(melody)
    i3 = (i1 + shift2) % len(melody)
    notes.append((melody[i1], melody[i2], melody[i3]))
    print(notes[-1])

for _ in range(1):
    play(Events(notes, scale=scale),
        d.client,
        dur=2/2/2/2/1.5,
        voice_amps=[0.6, 1, 1]
        )


(4, 6, 14)
(6, 11, 6)
(11, 13, 4)
(13, 14, 13)
(14, 6, 11)
(6, 4, 6)
(4, 13, 14)
(13, 11, 13)
(11, 6, 4)
(6, 14, 6)
(14, 13, 11)
(13, 4, 13)


IndexError: list index out of range

In [ ]:
# 14 edo
# https://en.xen.wiki/w/14edo
# xenharmonic appeal
# three types of 3rd
# has a recognizable 4th and 5th
# three types of 6th
# triad-rich 9-note MOS scale of 5L 4s
# 7 of 9 notes are tonic to a subminor, supermajor, and/or neutral triad

# Chord names
# 0-4-8 = C E G = C = C or C perfect
# 0-3-8 = C vE G = Cv = C down
# 0-5-8 = C ^E G = C^ = C up
# 0-4-7 = C E vG = C(v5) = C down-five
# 0-5-9 = C ^E ^G = C^(^5) = C up up-five
# 0-4-8-12 = C E G B = C7 = C seven
# 0-4-8-11 = C E G vB = C,v7 = C add down-seven
# 0-3-8-12 = C vE G B = Cv,7 = C down add seven
# 0-3-8-11 = C vE G vB = Cv7 = C down-seven

# scales
# titanium[9]
# 0LLsLsLsLs
#       1111
# 0245780134
# triads:
# x.x.x
# x.x..x
# x..x.x
# .x.x..x
# .x..x.x

# miniature composition:
# jazz chords
# triad pairs
# Chromatic Etude for 10 EDO Piano No. 1 “Intervals”
# Hideya A
# phasing
# micros album
# beat-tracker
# form building experiments, operators
# modulating synths, percussion into pitch
# PortaFM
# layered + looped music
# operations: add layer, remove layer, modulate

# TODO: create some kind of system of prototyping
# of loops, layers, generators and constraints/dependencies
# which will be solved like a puzzle
# similar to ffugue, i guess, but more general
# and more stochastic / unsound as a system

from domblar.ed import ED
chroma_steps = 14
d.chroma = ED(chroma_steps)
# steps = 7
# d.scale = mos(steps, 3, d.chroma)
bar_len = 8

def gen_motif():
    # we actually will create several voices out of a single chord
    pass

# how i do it right now in 12edo
# define scale with 7 notes
# then we can define chord progression
# with smooth voice-leading
# after that we can extract voices

voices = gen_motif(len=bar_len, voices=4)
composition = layer_voices(voices, order=[0, 1, 2, 3])
for i in range(4):
    composition.mutate_voice(i)

def set_synths():
    for i in range(synth_count):
        d.set_synth(i, 'jpiano')
set_synths()

# TODO: do some discovery:
# - intervals
#   3s = 234c = 8/7
#   4s = 312c = 6/5, min 3rd
#   5s = 390c = 5/4, maj 3rd
#   6s = 468c = 21/16
#   7s = 546c = 11/8
#   8s = 624c = 10/7
#   9s = 702c = 3/2
#   10s = 780c = 11/7
#   11s = 858c = 18/11
#   12s = 936c = 12/7
#   13s = 1014c = 9/5
#   14s = 1092c = 15/8
#   15s = 1170c
#   16s = 1248c
#   17s = 1326c = 15/7
#   18s ~= 16/7 (27 cents error)

# 7:8:10:11:12:15:18:27
# (0, 3, 8, 10, 12, 17, 21, 30)
# 3522549
# only 6s = 21/16 is missing

# a lot of /7 intervals

# - (sub-)scales (?), trichords, tetrachords/jins, pentatonic, ...

# - chords

# - chord progressions, modulations

# - phrases, motifs, melodies

# TODO: tonnetz should work!

# TODO: give note names
# TODO: draw lattice

# TODO: percussion

# TODO: check JI partials; which fit good?

# apply this knowledge to create composition, e. g.:
# - have a harmonic plan
# ...

# notes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
# notes = [0, 3, 5, 9]
# notes = [0, 8]
notes = [(0, 4, 9), (-1, 4, 8)]
# notes = [(0, 4, 9), (0, 5, 9)]


notes = [(0, 4, 9, -4)]
notes = [0, 4, 9, -4, -2]
notes = [-4, 1, 0, 4, 9]

# TODO: discovery - for chord with indices find some interpretation (e. g. in JI)
# TODO: (-3, 3, 7, 12)

# 3s=8/7, 7s=11/8
notes = [0, 2, 3, 4, 6, 7, 8, 10, 11, 12, 14, 15]
notes = [9, 13, 0, 9]
notes = [(0, 9, 13)]
notes = [(-14, 1, 5, 8), (-15, 0, 5, 9)]

notes = []
cur_note = 0
for jump in [2,2,3,2,2,2,3,2]:
    notes.append((cur_note, cur_note + 5, cur_note + jump * 4))
    cur_note += jump

# from domblar.transformations import iterate_voice_leadings
# notes = iterate_voice_leadings(
#     (0, 2, 7, 11),
#     [-2, 10, -2, -2],
#     [0, 2, 3, 1],
#     16)
# print(notes)

j1 = 9
notes = [0, (j1,j1+4), (j1,j1+4),
         -6, (j1,j1+4), (j1,j1+4),
         -3, (j1,j1+4), (j1,j1+4),
         -2, (j1,j1+4), (j1,j1+4),]
# notes = [(0,j1,j1+4),
#          (-6,j1,j1+4),
#          (-3,j1,j1+4),
#          (-2,j1,j1+4),]

def gen_chord(steps, note=0):
    chord = [note]
    for s in steps:
        chord.append(chord[-1] + s)
    return tuple(chord)

# 7:8:10:11:12:15:18:27 chord = (0, 3, 8, 10, 12, 17, 21, 30)
notes = [(-3, 3, 7, 12), (0, 3, 8, 10, 12, 17, 21, 30)]

# TODO: tonnnetz
# - add it from live_coding_studies1 to code
# - then check here
# - (also maybe check 12edo examples)

# notes = [(0, 3, 10), (8, 0, 12, 17)]
# notes = [(-14, 1, 5, 8), (-15, 0, 5, 9)]

notes = [9, 4, 9, 0, 9, 13]
# notes = [(0, 4, 9, 13)]

notes = [0, 3, 8, 10, 12, 17, 21, 30, 21, 17, 12, 10, 8, 3]
# TODO: overtones of the overtones + voice-leading
# 3522549
# 3322232223222
notes = [0,3,6,8,10,12,15,17,19,21,24,26,28,30,33,35]
notes = [n for n in notes]
notes = [(0,3,8), (3,8,12), (10,15,19), (8,12,17)]
# notes = [(0,3,8), (3,8,12), (8,12,17)]

def perc_parse(beat, val=0):
    if beat == 'x':
        return val
    return beat

def mix(streams):
    return list(zip(*streams))


# notes = [0, 3, 8, 10, 12, 17, 21, 30]
# ns1 = [0,3,6,8,10,12,15,17,19,21,24,26,28,30]
# ns2 = [-9, -6, -3, -1, 1, 3, 6, 8, 10, 12, 15, 17, 19, 21]
# notes = mix([ns1, ns2])

#  7 :8 :10 :11 :12 :15 :18 :27 chord
# (0, 3,  8, 10, 12, 17, 21, 30)
# notes = [(3, 10, 17)]
# notes = [(3, 12, 30)]
# notes = [(0, 8, 10)]
# notes = [(0, 8, 12)]
# notes = [(3, 8, 12)]  # 3 maj
# notes = [(0, 5, 9), (0, 8, 12),
#          (3, 8, 12), (3, 12, 17), (3, 11, 15),
#          (8, 12, 17), (8, 17, 21)]
# notes = [(3, 11, 15),
        #  (8, 12, 17), (3, 17, 21)]
# notes = [(0, 8, 12, 21)]
# notes = [(0, 6, 10)]  # sounds minor!
# notes = [(0, 4, 9), (0, 6, 10), (6, 10, 15)]
# notes = [(0, 6, 10, 15), (3, 8, 12, 17),
#          (6, 10, 15, 19), '.', (8, 12, 17, 21),
#          (10, 15, 19, 24), (12, 17, 21, 26)]
# notes = [(0, 8, 15, 21), (3, 10, 17, 24)]
# notes = [(0, 8, 15, 21), 15, 8, 21, (3, 10, 17, 24), 17, 10, 3]
# notes = [0,3,'.',6,8,10,6]
# notes = [(0, 3, 8, 10, 12, 17, 21, 30)]
# notes = [tuple((x + 10) for x in notes[0])]
# while True:
for _ in range(1):
    play(Events(notes, scale=scale),
        d.client,
        # dur=1.,
        # dur=.5,
        dur=.25,
        #  synth_idx=[0],
        )
    # finetune(1)

# TODO: pentatonic, 32424, 42342
# TODO: pentatonic 1⁄1–19⁄16–21⁄16–3⁄2–7⁄4
# TODO: pentatonic intervals between scale steps are 8:7–7:6–9:8–8:7–7:6

# Molten Pelog
# 155.990
# 545.965
# 701.955
# 857.945
# 1247.920

# Molten Slendro
# 233.985
# 467.970
# 701.955
# 935.940
# 1169.925

# exploring Kostka-Payne chart
# https://i.stack.imgur.com/JfJbu.png
# iii -> vi -> S:[IV -> ii] -> D:[V <-> vii] -> I .
# (or maybe just vii -sometimes> V)
# iii -> S -> I
# D -> vi .

# chord progressions (mini-notation) idea
# H = Harmony
# g = H()
# sub = H() > 'IV' > 'ii'
# dom = H() > 'V' > 'vii' > 'V'
# g > 'iii' > 'vi' > sub > dom > ['Ij.', 'vi.']
# g > 'iii' > sub > 'I'
# d <[
#   g
# ]
# how to add modulations? with another grammar?

# but what about 9edf?
# either i need scales
# or something different

# chord progressions for Tonnetz?
# but it's just a movement on 2d hexagonal grid

# another idea is to generate cadences
# and non-cadences

ModuleNotFoundError: No module named 'domblar'

In [72]:
from domblar.ed import ED

# for i in range(2):
d.set_synth(0, 'jpiano')
d.set_synth(1, 'marimba')


# scale = ED(12)
# phrase1 = [4, 11, 14] * 2  # steps: 7, 10
# phrase2 = [6, 13] * 3  # steps: 2, 7

scale = ED(14)
# titanium = subscale(scale, 'LLsLsLsLs')
# triads = discover_triads(titanium)
# tetrads = discover_tetrads(titanium)
triad_shapes = [
    (0, 4, 8),  # C E G = C = C or C perfect
    (0, 3, 8),  # C vE G = Cv = C down
    (0, 5, 8),  # C ^E G = C^ = C up
    (0, 4, 7),  # C E vG = C(v5) = C down-five
    (0, 5, 9),  # C ^E ^G = C^(^5) = C up up-five
]
tetrad_shapes = [
    (0, 4, 8, 12),  # C E G B = C7 = C seven
    (0, 4, 8, 11),  # C E G vB = C,v7 = C add down-seven
    (0, 3, 8, 12),  # C vE G B = Cv,7 = C down add seven
    (0, 3, 8, 11),  # C vE G vB = Cv7 = C down-seven
]

# phrase1 = [4, 12, 15] * 2  # steps: 8, 13
# phrase2 = [6, 14] * 3  # steps: 2, 8
# 0245780134
# +4
# 4689124578

melody = []
for i in range(len(phrase1)):
    melody.append(phrase1[i])
    melody.append(phrase2[i])

notes = []
for i1 in range(len(melody)):
    i2 = (i1 + 1) % len(melody)
    notes.append((melody[i1], melody[i2]))

# TODO: how to build it more modularly?
# c = Composition2DGrid(voices=4, bar=8)
# c[0].add(melody)
# c.render()
# play(c, d.client,
#      dur=2/2/2/2/1.5,
#      voice_amps=[0.6, 1]
# )

for _ in range(1):
    play(Events(notes, scale=scale),
        d.client,
        dur=2/2/2/2/1.5,
        voice_amps=[0.6, 1]
        )


In [ ]:
# mutations explorations

# change mode
# input: 2 scales + melody in scale1
# output: melody in scale 2, with smoothest voice leading

In [1]:
# 14 edo
# https://en.xen.wiki/w/14edo
# xenharmonic appeal
# three types of 3rd
# has a recognizable 4th and 5th
# three types of 6th
# triad-rich 9-note MOS scale of 5L 4s
# 7 of 9 notes are tonic to a subminor, supermajor, and/or neutral triad

# Chord names
# 0-4-8 = C E G = C = C or C perfect
# 0-3-8 = C vE G = Cv = C down
# 0-5-8 = C ^E G = C^ = C up
# 0-4-7 = C E vG = C(v5) = C down-five
# 0-5-9 = C ^E ^G = C^(^5) = C up up-five
# 0-4-8-12 = C E G B = C7 = C seven
# 0-4-8-11 = C E G vB = C,v7 = C add down-seven
# 0-3-8-12 = C vE G B = Cv,7 = C down add seven
# 0-3-8-11 = C vE G vB = Cv7 = C down-seven

# scales
# titanium[9]
# 0LLsLsLsLs
#       1111
# 0245780134
# triads:
# x.x.x
# x.x..x
# x..x.x
# .x.x..x
# .x..x.x

# miniature composition:
# jazz chords
# triad pairs
# Chromatic Etude for 10 EDO Piano No. 1 “Intervals”
# Hideya A
# phasing
# micros album
# beat-tracker
# form building experiments, operators
# modulating synths, percussion into pitch
# PortaFM
# layered + looped music
# operations: add layer, remove layer, modulate

# TODO: create some kind of system of prototyping
# of loops, layers, generators and constraints/dependencies
# which will be solved like a puzzle
# similar to ffugue, i guess, but more general
# and more stochastic / unsound as a system

from domblar.ed import ED
chroma_steps = 14
d.chroma = ED(chroma_steps)
# steps = 7
# d.scale = mos(steps, 3, d.chroma)
bar_len = 8

def gen_motif():
    # we actually will create several voices out of a single chord
    pass

# how i do it right now in 12edo
# define scale with 7 notes
# then we can define chord progression
# with smooth voice-leading
# after that we can extract voices

voices = gen_motif(len=bar_len, voices=4)
composition = layer_voices(voices, order=[0, 1, 2, 3])
for i in range(4):
    composition.mutate_voice(i)

def set_synths():
    for i in range(synth_count):
        d.set_synth(i, 'jpiano')
set_synths()

# TODO: do some discovery:
# - intervals
#   3s = 234c = 8/7
#   4s = 312c = 6/5, min 3rd
#   5s = 390c = 5/4, maj 3rd
#   6s = 468c = 21/16
#   7s = 546c = 11/8
#   8s = 624c = 10/7
#   9s = 702c = 3/2
#   10s = 780c = 11/7
#   11s = 858c = 18/11
#   12s = 936c = 12/7
#   13s = 1014c = 9/5
#   14s = 1092c = 15/8
#   15s = 1170c
#   16s = 1248c
#   17s = 1326c = 15/7
#   18s ~= 16/7 (27 cents error)

# 7:8:10:11:12:15:18:27
# (0, 3, 8, 10, 12, 17, 21, 30)
# 3522549
# only 6s = 21/16 is missing

# a lot of /7 intervals

# - (sub-)scales (?), trichords, tetrachords/jins, pentatonic, ...

# - chords

# - chord progressions, modulations

# - phrases, motifs, melodies

# TODO: tonnetz should work!

# TODO: give note names
# TODO: draw lattice

# TODO: percussion

# TODO: check JI partials; which fit good?

# apply this knowledge to create composition, e. g.:
# - have a harmonic plan
# ...

# notes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
# notes = [0, 3, 5, 9]
# notes = [0, 8]
notes = [(0, 4, 9), (-1, 4, 8)]
# notes = [(0, 4, 9), (0, 5, 9)]


notes = [(0, 4, 9, -4)]
notes = [0, 4, 9, -4, -2]
notes = [-4, 1, 0, 4, 9]

# TODO: discovery - for chord with indices find some interpretation (e. g. in JI)
# TODO: (-3, 3, 7, 12)

# 3s=8/7, 7s=11/8
notes = [0, 2, 3, 4, 6, 7, 8, 10, 11, 12, 14, 15]
notes = [9, 13, 0, 9]
notes = [(0, 9, 13)]
notes = [(-14, 1, 5, 8), (-15, 0, 5, 9)]

notes = []
cur_note = 0
for jump in [2,2,3,2,2,2,3,2]:
    notes.append((cur_note, cur_note + 5, cur_note + jump * 4))
    cur_note += jump

# from domblar.transformations import iterate_voice_leadings
# notes = iterate_voice_leadings(
#     (0, 2, 7, 11),
#     [-2, 10, -2, -2],
#     [0, 2, 3, 1],
#     16)
# print(notes)

j1 = 9
notes = [0, (j1,j1+4), (j1,j1+4),
         -6, (j1,j1+4), (j1,j1+4),
         -3, (j1,j1+4), (j1,j1+4),
         -2, (j1,j1+4), (j1,j1+4),]
# notes = [(0,j1,j1+4),
#          (-6,j1,j1+4),
#          (-3,j1,j1+4),
#          (-2,j1,j1+4),]

def gen_chord(steps, note=0):
    chord = [note]
    for s in steps:
        chord.append(chord[-1] + s)
    return tuple(chord)

# 7:8:10:11:12:15:18:27 chord = (0, 3, 8, 10, 12, 17, 21, 30)
notes = [(-3, 3, 7, 12), (0, 3, 8, 10, 12, 17, 21, 30)]

# TODO: tonnnetz
# - add it from live_coding_studies1 to code
# - then check here
# - (also maybe check 12edo examples)

# notes = [(0, 3, 10), (8, 0, 12, 17)]
# notes = [(-14, 1, 5, 8), (-15, 0, 5, 9)]

notes = [9, 4, 9, 0, 9, 13]
# notes = [(0, 4, 9, 13)]

notes = [0, 3, 8, 10, 12, 17, 21, 30, 21, 17, 12, 10, 8, 3]
# TODO: overtones of the overtones + voice-leading
# 3522549
# 3322232223222
notes = [0,3,6,8,10,12,15,17,19,21,24,26,28,30,33,35]
notes = [n for n in notes]
notes = [(0,3,8), (3,8,12), (10,15,19), (8,12,17)]
# notes = [(0,3,8), (3,8,12), (8,12,17)]

def perc_parse(beat, val=0):
    if beat == 'x':
        return val
    return beat

def mix(streams):
    return list(zip(*streams))


# notes = [0, 3, 8, 10, 12, 17, 21, 30]
# ns1 = [0,3,6,8,10,12,15,17,19,21,24,26,28,30]
# ns2 = [-9, -6, -3, -1, 1, 3, 6, 8, 10, 12, 15, 17, 19, 21]
# notes = mix([ns1, ns2])

#  7 :8 :10 :11 :12 :15 :18 :27 chord
# (0, 3,  8, 10, 12, 17, 21, 30)
# notes = [(3, 10, 17)]
# notes = [(3, 12, 30)]
# notes = [(0, 8, 10)]
# notes = [(0, 8, 12)]
# notes = [(3, 8, 12)]  # 3 maj
# notes = [(0, 5, 9), (0, 8, 12),
#          (3, 8, 12), (3, 12, 17), (3, 11, 15),
#          (8, 12, 17), (8, 17, 21)]
# notes = [(3, 11, 15),
        #  (8, 12, 17), (3, 17, 21)]
# notes = [(0, 8, 12, 21)]
# notes = [(0, 6, 10)]  # sounds minor!
# notes = [(0, 4, 9), (0, 6, 10), (6, 10, 15)]
# notes = [(0, 6, 10, 15), (3, 8, 12, 17),
#          (6, 10, 15, 19), '.', (8, 12, 17, 21),
#          (10, 15, 19, 24), (12, 17, 21, 26)]
# notes = [(0, 8, 15, 21), (3, 10, 17, 24)]
# notes = [(0, 8, 15, 21), 15, 8, 21, (3, 10, 17, 24), 17, 10, 3]
# notes = [0,3,'.',6,8,10,6]
# notes = [(0, 3, 8, 10, 12, 17, 21, 30)]
# notes = [tuple((x + 10) for x in notes[0])]
# while True:
for _ in range(1):
    play(Events(notes, scale=scale),
        d.client,
        # dur=1.,
        # dur=.5,
        dur=.25,
        #  synth_idx=[0],
        )
    # finetune(1)

# TODO: pentatonic, 32424, 42342
# TODO: pentatonic 1⁄1–19⁄16–21⁄16–3⁄2–7⁄4
# TODO: pentatonic intervals between scale steps are 8:7–7:6–9:8–8:7–7:6

# Molten Pelog
# 155.990
# 545.965
# 701.955
# 857.945
# 1247.920

# Molten Slendro
# 233.985
# 467.970
# 701.955
# 935.940
# 1169.925

# exploring Kostka-Payne chart
# https://i.stack.imgur.com/JfJbu.png
# iii -> vi -> S:[IV -> ii] -> D:[V <-> vii] -> I .
# (or maybe just vii -sometimes> V)
# iii -> S -> I
# D -> vi .

# chord progressions (mini-notation) idea
# H = Harmony
# g = H()
# sub = H() > 'IV' > 'ii'
# dom = H() > 'V' > 'vii' > 'V'
# g > 'iii' > 'vi' > sub > dom > ['Ij.', 'vi.']
# g > 'iii' > sub > 'I'
# d <[
#   g
# ]
# how to add modulations? with another grammar?

# but what about 9edf?
# either i need scales
# or something different

# chord progressions for Tonnetz?
# but it's just a movement on 2d hexagonal grid

# another idea is to generate cadences
# and non-cadences

ModuleNotFoundError: No module named 'domblar'

In [4]:
# TODO: add support for C-Thru Music AXiS-49

In [5]:
# TODO: smth similar to Stephane Picq, Dune - Water

In [6]:
d.set_synth(0, 'sdrum2')
notes1 = '.xx.xxxx'
notes1 = [perc_parse(n) for n in notes1]
d.set_synth(1, 'bdrum_ok')
notes2 = 'x...x...'
notes2 = [perc_parse(n, -20) for n in notes2]
d.set_synth(2, 'dochat2')
notes3 = '...x.x..'
notes3 = [perc_parse(n, -10) for n in notes3]
notes = mix([notes1, notes2, notes3])

# while True:
for _ in range(2):
    play(Events(notes, scale=scale),
        d.client,
        dur=0.125,
        # dur=1.,
        # dur=3/2/2,
        # dur=.5,
        # dur=.25,
        )

In [7]:
d.set_synth(0, 'sdrum2')
notes1 = '....x..x.x..x..x'
notes1 = [perc_parse(n) for n in notes1]
d.set_synth(1, 'bdrum_ok')
notes2 = 'x.x.x.x.x.x.x.x.'
notes2 = [perc_parse(n, -20) for n in notes2]
d.set_synth(2, 'dochat2')
notes3 = 'x.x.......xx....'
notes3 = [perc_parse(n, -15) for n in notes3]
notes = mix([notes1, notes2, notes3])

# while True:
for _ in range(1):
    play(Events(notes, scale=scale),
        d.client,
        dur=0.1,
        # dur=1.,
        # dur=3/2/2,
        # dur=.5,
        # dur=.25,
        )

In [8]:
# voice-leading?
# problem: we don't have note octave-transpositions,
# we don't have chord inversions
# but, nevertheless
# we can try to voice-lead a chord into another
# if we somehow specify the desired shapes
# or set of shapes
# we can also gradually change the chords

In [10]:
for i in range(6):
    d.set_synth(i, 'jpiano')
d.set_synth(0, 'marimba2')
d.set_synth(3, 'marimba2')

# notes = [0, 9, 11, 15, 11]
# notes = [3, 8, 10, 11, 10]
# notes = [(3, 8, 10, 15), (0, 5, 9)]
notes = [(4, 9, 13), '.',
         (0, 5, 9), '.',#(5, 0, 9, 13),
         (7, 12, 16, 20),
         (4, 9, 13, 17, 22),
         (0, 5, 9, 13, 20, 24)]
# notes = [(4, 9, 13, 17, 22)]
# notes = [(0, 4, 7)]
        #  (5, 9, 18),
        #  (9, 18, 23)]
# notes = [(0, 4, 9)]

# while True:
for _ in range(1):
    play(Events(notes, scale=scale),
        d.client,
        # dur=0.125,
        # dur=1.,
        dur=3/2,
        voice_amps=[1,1,1,1,1,1]
        # dur=.5,
        # dur=.25,
        )


KeyboardInterrupt: 

In [11]:
for i in range(8):
    if i < 6:
        d.set_synth(i, 'marimba')
    else:
        d.set_synth(i, 'marimba2')
d.set_synth(6, 'dune_violin2')
d.set_synth(7, 'dune_violin2')
d.set_synth(8, 'sdrum2')
# d.set_synth(9, 'bdrum_ok')

ns1 = [4, 9, (13, 17, 22), 17, 22, 17]
ns2 = [2, 6, (11, 15, 19), 15, 19, 15]
ns3 = ['(".", ".", ".", 0, 5, 9, 13, 18):3']
ns4 = ['(".", ".", ".", -1, 4, 8, 12, 17):3']
ns5 = ['(".", ".", ".", ".", ".", ".", ".", ".", 0):1']
notes = ns1 + ns2 + ns3 + ns4 + ns3 + ns5

# d.set_synth(0, 'sdrum2')
# notes1 = '.xx.xxxx'
# notes1 = [perc_parse(n) for n in notes1]
# d.set_synth(1, 'bdrum_ok')
# notes2 = 'x...x...'
# notes2 = [perc_parse(n, -20) for n in notes2]

d.rec()
# while True:
for _ in range(1):
    play(Events(notes, scale=scale),
        d.client,
        # dur=0.125,
        # dur=1.,
        dur=3/2/2/2/1.5,
        # voice_amps=[1,1,1,1,1,1,1,1]
        # dur=.5,
        # dur=.25,
        )
d.stop_rec()

In [ ]:
# ns1 = [4, 9, (13, 17, 22), 17, 22, 17]
# i want to expand on this example
# add grammar/notation for manipulations:
# semi-automatic mutations
# semi-automatic gestures
# add percussion

# pitches: 4,9,13,17,22
# diffs: 5, 4, 4, 5 - so it's almost some kind of MOS scale


In [ ]:
# draw random 3-4 notes chords from a collection of notes?
# with specified minimum distance between each consecutive pair of notes
# so that chords would have a nice spread

# alternative: treat 9 steps as equivalence
# assume, say, 3222 as MOS scale
# but this doesn't work: we get inversion 0,5,9 -> 9,5,9


In [706]:
finetune(0, 0)
finetune(2, 0)

In [ ]:
from domblar.chord_theory import Chords

dur = 0.33
[Chords()  # TODO: Chords(ctx)
    .synths([0, 1, 2, 3])
    .seq(4, [-3, 2],
        #  TODO:
        #  mods='r1:4/4 f6:5/6',  # r=rewrite, f=modulate_from
         length=5,
         edo=12,
         scale=[0,2,4,5,7,9,11]
         )
    .voicing('0_ 1 2_')
    .q('quartal')
    .g([
        '0:4*2',
        '3__:2 .:4 2__:2',
        '1 . 1 . 1 . 1 .',  # TODO: '{1 .}*4',
        '. 2 . 3 . 2 . 3',  # TODO: '{. 2 . 3}*2',
        ])
    .dur(dur)  # FIXME: better to replace with beat-dur
    .sus(dur * 0.95)
]

[[[-7, 11, 4, 21], [-7, 12, 4, 23], [-5, 14, 4, 21], [-1, 14, 4, 21], [0, 14, 5, 19]]]